In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 전처리 파이프 라인
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드
train_dataset = datasets.ImageFolder(root='pokemon-dataset-1000/train', transform=transform)
val_dataset = datasets.ImageFolder(root='pokemon-dataset-1000/val', transform=transform)
test_dataset = datasets.ImageFolder(root='pokemon-dataset-1000/test', transform=transform)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
import timm

# 모델 초기화
model = timm.create_model('mobilenetv3_large_100', pretrained=True, num_classes=len(train_dataset.classes))

# 모델을 GPU로 이동 (가능한 경우)
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# # Apple Silicon
# # GPU 설정
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# # 모델을 GPU 또는 CPU로 이동
# model = model.to(device)

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 훈련 및 검증
for epoch in range(50):  # 에포크 수
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    print(f"Epoch {epoch+1}, Training Loss: {train_loss/len(train_loader)}")

    # 검증
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct/total:.2f}%")

In [ ]:
# 테스트 데이터로 평가
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct/total:.2f}%")

# 모델 저장
torch.save(model.state_dict(), "trained_model.pth")

In [ ]:
#모델 불러오기

import torch
import timm

# 모델 초기화
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('mobilenetv3_large_100', pretrained=False, num_classes=len(train_dataset.classes))
model = model.to(device)

# 저장된 가중치 불러오기
model.load_state_dict(torch.load("trained_model.pth", map_location=device))

# 모델 평가 모드 설정
model.eval()

print("Model weights successfully loaded!")


In [ ]:
import random
import matplotlib.pyplot as plt
import torch

# 랜덤 샘플 100개 추출
num_samples = 100
random_indices = random.sample(range(len(test_loader.dataset)), num_samples)

# 테스트 데이터셋에서 랜덤 샘플 가져오기
images, labels = [], []
for idx in random_indices:
    img, lbl = test_loader.dataset[idx]
    images.append(img)
    labels.append(lbl)

# 이미지를 모델 입력 형식으로 변환
images_tensor = torch.stack(images).to(device)
labels_tensor = torch.tensor(labels).to(device)

# 모델 예측
model.eval()
with torch.no_grad():
    outputs = model(images_tensor)
    _, predicted = torch.max(outputs, 1)

# 결과 시각화 (10x10 그리드로 출력)
fig, axes = plt.subplots(10, 10, figsize=(15, 15))
axes = axes.ravel()

# 정확도 계산용 변수
correct_count = 0

for i in range(num_samples):
    # 이미지를 [0, 1] 범위로 변환
    img = images[i].permute(1, 2, 0).cpu().numpy()
    img = (img - img.min()) / (img.max() - img.min())
    
    # 시각화
    axes[i].imshow(img)
    pred_label = predicted[i].item()
    true_label = labels[i]
    
    # 정확도 계산
    if pred_label == true_label:
        correct_count += 1  # 맞춘 경우 카운트 증가
    
    axes[i].set_title(f"Pred: {pred_label}\nTrue: {true_label}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# 정확도 계산 및 출력
accuracy = correct_count / num_samples * 100
print(f"Accuracy: {accuracy:.2f}%")

# 바 차트로 정확도 표현
plt.figure(figsize=(6, 4))
plt.bar(["Correct", "Incorrect"], [correct_count, num_samples - correct_count], color=["green", "red"])
plt.title("Model Prediction Accuracy")
plt.ylabel("Number of Images")
plt.xticks(rotation=0)
plt.show()

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
import os

# 데이터셋 로드 및 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root='pokemon-dataset-1000/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# 모델 초기화
import timm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('mobilenetv3_large_100', pretrained=True, num_classes=len(train_dataset.classes))
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 에포크 수 리스트
epochs_list = [1, 10, 30, 50, 70, 100, 200]

# 결과 저장용 리스트
results = []

# 모델 저장 디렉토리 생성
os.makedirs("saved_models", exist_ok=True)

# 각 에포크에 대해 모델 학습
for num_epochs in epochs_list:
    print(f"Training model for {num_epochs} epochs...")
    start_time = time.time()  # 시작 시간 기록

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

    end_time = time.time()  # 종료 시간 기록
    elapsed_time = end_time - start_time  # 학습에 걸린 시간
    print(f"Completed {num_epochs} epochs in {elapsed_time:.2f} seconds.")

    # 결과 저장
    results.append(f"Epochs: {num_epochs}, Time: {elapsed_time:.2f} seconds")

    # 학습된 모델 저장
    model_path = f"saved_models/model_{num_epochs}_epochs.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

# 결과를 텍스트 파일에 저장
with open("training_times.txt", "w") as file:
    file.write("\n".join(results))

print("Training times and models saved.")

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
import os

# 데이터셋 로드 및 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root='pokemon-dataset-1000/train', transform=transform)
val_dataset = datasets.ImageFolder(root='pokemon-dataset-1000/val', transform=transform)
test_dataset = datasets.ImageFolder(root='pokemon-dataset-1000/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# 모델 초기화
import timm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('mobilenetv3_large_100', pretrained=True, num_classes=len(train_dataset.classes))
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 에포크 수 리스트
epochs_list = [1, 10, 30, 50, 70, 100, 200]

# 결과 저장용 리스트
results = []

# 모델 저장 디렉토리 생성
os.makedirs("saved_models", exist_ok=True)

# 각 에포크에 대해 모델 학습
for num_epochs in epochs_list:
    print(f"Training model for {num_epochs} epochs...")
    start_time = time.time()  # 시작 시간 기록

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

    end_time = time.time()  # 종료 시간 기록
    elapsed_time = end_time - start_time  # 학습에 걸린 시간
    print(f"Completed {num_epochs} epochs in {elapsed_time:.2f} seconds.")

    # 테스트 데이터로 평가
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    # 결과 저장
    results.append(f"Epochs: {num_epochs}, Time: {elapsed_time:.2f} seconds, Test Accuracy: {test_accuracy:.2f}%")

    # 학습된 모델 저장
    model_path = f"saved_models/model_{num_epochs}_epochs.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

# 결과를 텍스트 파일에 저장
with open("training_times_and_accuracy.txt", "w") as file:
    file.write("\n".join(results))

print("Training times, test accuracies, and models saved.")